In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

## 3.2 Finite difference method

$$\begin{equation}
\frac{\partial C}{\partial t^*} = \frac{C_i^{n+1} - C_i^{n}}{\Delta t}\\
\frac{\partial C}{\partial S} = \frac{C_{i+1}^{n} - C_{i-1}^{n}}{2\Delta S}\\
\frac{\partial^2 C}{\partial S^2} = \frac{C_{i+1}^{n} - 2C_i^{n} + C_{i-1}^{n}}{(\Delta S)^2}\\
\end{equation}$$

the discretized Black-Scholes equation:
$$\frac{C_i^{n+1} - C_i^{n}}{\Delta t} - \frac{1}{2} \sigma^2 S^2 \frac{C_{i+1}^{n} - 2C_i^{n} + C_{i-1}^{n}}{(\Delta S)^2} - rS \frac{C_{i+1}^{n} - C_{i-1}^{n}}{2\Delta S} + rC_i^{n} = 0 $$

since $S=i \Delta S$, we can rearrange the above equation as:

$$\frac{C_i^{n+1} - C_i^{n}}{\Delta t} - \frac{1}{2} \sigma^2 i^2 (C_{i+1}^{n} - 2C_i^{n} + C_{i-1}^{n}) - \frac{1}{2} ri (C_{i+1}^{n} - C_{i-1}^{n}) + rC_i^{n} = 0 $$

the option price at the next time step can be expressed as:
$$\begin{equation}
C_i^{n+1} = \frac{1}{2}(\sigma^2 i^2 \Delta t-ri\Delta t)C_{i-1}^n + (1 - \sigma^2 i^2 \Delta t -r \Delta t) C_i^n + \frac{1}{2}(\sigma^2 i^2 \Delta t+ri\Delta t)C_{i+1}^n 
\end{equation}$$

In [84]:
from scipy.stats import norm
def Black_Scholes_Call(S, K, r, vol, tau):
    """ 
    Black Scholes Model for European Call
    """
    d1 = (np.log(S / K) + (r + ((vol**2)/2.)*tau)) / (vol*np.sqrt(tau))
    d2 = d1 - vol*np.sqrt(tau)
    V = S * norm.cdf(d1) - np.exp(-r*tau) * K * norm.cdf(d2)
    
    return V

In [381]:
sigma = 0.3
K = 110
S = 120
r = 0.04
T = 1
N = 6000# time points
M =  300# space points
smin = 0.0001
smax = S*2

dt = T/N       # time step
dx = np.log(smax-smin)/M
s = np.linspace(smin, smax, M)   # spatial grid (stock's price)
#s = np.clip(s, smin, np.log(smax))
#s = np.log(s)

# initial condition & boundary condition
V = s - K * math.e**-r
V = np.clip(V, 0, smax)

For this equation, we conclude that
\begin{align}
    \vec{a_{-1}} &= \frac{1}{2}\sigma^{2}\frac{\Delta\tau}{\Delta X^{2}} - \frac{\Delta\tau}{2\Delta X}(r-\frac{1}{2}\sigma^{2}) \\
    \vec{a_{0}} &=  1- \sigma^{2}\frac{\Delta\tau}{\Delta X^{2}} - r\Delta\tau\\
    \vec{a_{+1}} &=  \frac{\Delta\tau}{2\Delta X}(r-\frac{1}{2}\sigma^{2})+\frac{1}{2}\sigma^{2}\frac{\Delta\tau}{\Delta X^{2}}
\end{align}

In [382]:
up = ((0.5*(sigma**2) * dt / dx**2)    -    ((r-0.5*(sigma**2)) *dt / (2*dx)))
mid = (1-((sigma**2) * dt / dx**2) - r * dt)
down = ((0.5*(sigma**2) * dt / dx**2)    +    ((r-0.5*(sigma**2)) *dt / (2*dx)))

In [383]:
print(up, mid, down)

0.005629407999542349 0.9887573249113907 0.005606600422400312


In [384]:
V_new = np.zeros(M)

In [385]:
for j in range(N+1):
    for i in range(1,M-1):
        #print(up * V[i-1] + mid * V[i] + down * V[i+1])
        V_new[i] = up * V[i-1] + mid * V[i] + down * V[i+1]
    
    V_new[0] = V[0]
    V_new[-1] = V[-1]
    #V_new = np.clip(V_new, 0, smax)
    #print(j, V)
    V = np.copy(V_new)
    #print(j, V)
    
V_new[int(len(V_new)/2)]

In [316]:
Black_Scholes_Call(S, K, r, sigma, 1)

21.788808338829327

In [ ]:
%matplotlib inline
import numpy
import matplotlib.pyplot as pyplot
from matplotlib import rcParams
rcParams['figure.dpi'] = 100
rcParams['font.size'] = 16
rcParams['font.family'] = 'StixGeneral'

T = 0.25       # expiry time
r = 0.1        # no-risk interest rate
sigma = 0.4    # volatility of underlying asset
E = 10.        # exercise price
S_max = 4*E    # upper bound of price of the stock (4*E)

In [ ]:
def FTCS(C, N, M, dt, r, sigma):
    """using forward-time central-space scheme to solve the Black-Scholes equation for the call option price
    
    Arguments:
        C:       array of the price of call option
        N:       total number of time steps
        M:       total number of spatials grids
        dt:      time step
        r:       no-risk interest rate
        sigma:   volatility of the stock
    
    Returns:
        C:       array of the price of call option
    """
    index = numpy.arange(1,M)
    for n in range(N):

        C[1:-1] = 0.5 * (sigma**2 * index**2 * dt - r*index*dt) * C[0:-2] \
             +       (1 - sigma**2* index**2 *dt - r*dt) * C[1:-1]   \
             + 0.5 * (sigma**2 * index**2 * dt + r*index*dt) * C[2:]
    return C

In [ ]:
N = 2000       # number of time steps 
M = 200        # number of space grids
dt = T/N       # time step
s = numpy.linspace(0, S_max, M+1)   # spatial grid (stock's price)

# initial condition & boundary condition
C = s - E
C = numpy.clip(C, 0, S_max-E)

In [ ]:
numpy.clip(C, 0, S_max-E)

In [ ]:
C_exp = FTCS(C, N, M, dt, r, sigma)
print(f'the price of the call option should be around {C_exp[int(M/2)]}, \
if the current price of stock is 20 dollar.')

In [ ]:
C[int(len(C)/2)]

In [ ]:
index = numpy.arange(1,M)
for n in range(N):

    C[1:-1] = 0.5 * (sigma**2 * index**2 * dt - r*index*dt) * C[0:-2] \
        +       (1 - sigma**2* index**2 *dt - r*dt) * C[1:-1]   \
        + 0.5 * (sigma**2 * index**2 * dt + r*index*dt) * C[2:]
    

In [ ]:
pyplot.figure(figsize=(8,5), dpi=100)
pyplot.plot(s,C_exp,color='#20b2aa', ls='--', lw=3, label='FTCS');
pyplot.xlabel('Current Price of the Stock (S)')
pyplot.ylabel('Price of the call option (C)')
pyplot.legend(loc='upper left',prop={'size':15});